In [2]:
import requests
from scrapy.selector import Selector
import re
from requests_html import HTMLSession
import json
import time
from tqdm import tqdm

In [10]:
exclusion = ['\xa0']
regex_pattern_replacements = re.compile(*exclusion)

items = []
for page in tqdm(range(1, 79, 1)):
# Note: manually checked, theres only 78 pages as of 4 Feb 2024
    landing_page_response = HTMLSession().get(f"https://technode.global/category/onlocation/malaysia/?upage={page}")
    urls = Selector(landing_page_response).xpath('//h3[@class="t-entry-title h3 title-scale"]//@href').getall()
    titles = Selector(landing_page_response).xpath('//h3[@class="t-entry-title h3 title-scale"]/a').css('::text').getall()

    for i,x in enumerate(urls):
        contents = {}
        texts = []
        response = HTMLSession().get(x)
        articles = Selector(response).css('div.post-body div.row-container > div h2,p')
        for x in articles:
            header_text = x.css('h2::text')
            if header_text:
                header_text = re.sub(regex_pattern_replacements, '', header_text.get())
                texts.append(header_text)
            else:
                para_text = x.css('::text').get()
                if para_text == None: 
                    continue
                para_text = re.sub(regex_pattern_replacements, '', para_text)
                texts.append(para_text)
        texts = ''.join(texts[:-3]) # -3 because in every page got "Analysis, About, and TNGlobalTechNode EN" to be excluded
        texts = re.sub('\.(?!\s|\d|$)', '. ', texts)
        
        contents['url'] = urls[i]
        contents['page'] = page
        contents['title'] = titles[i]
        contents['contents'] = ''.join(texts)
        items.append(contents)
        with open('data.json', 'w', encoding='utf8') as f:
            f.write(json.dumps(items))
    time.sleep(2) 
# items

100%|██████████| 78/78 [11:54<00:00,  9.16s/it] 
